In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('uji.csv')
X = df.iloc[:, 2:51]   # Using 1:2 as indices will give us np array of dim (10, 1)
y = df.iloc[:, 51]

df.head()

,Source Port,Destination Port,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Flow Bytes/s,Flow Packets/s,Flow IAT Std,...,Idle Std,Flow ID_hash,Source IP_hash,Destination IP_hash,Timestamp_hash,Packet Length Std_hash,CWE Flag Count_0.0,CWE Flag Count_0,CWE Flag Count_SCAREWARE,Class_num
0,-0.210468,-0.368578,-0.212203,-0.182887,-0.018077,1.992640,-0.356094,-0.100220,-0.140149,0.112635,...,-0.121634,0.714752,-0.806277,-0.026111,-0.482692,0.345380,-0.052787,-0.214913,-0.002394,0
1,0.828401,-0.368578,-0.212203,-0.417931,-0.144807,1.938685,-0.356094,0.606939,-0.120963,-0.384509,...,-0.121634,0.974543,-0.091533,1.649509,-0.204585,1.717001,-0.052787,-0.214913,-0.002394,0
2,-2.027754,2.839532,-0.212203,-0.260828,-0.266317,-0.611576,-0.356094,1.191516,1.032915,-0.385460,...,-0.121634,-0.544263,-0.642525,1.795664,1.477229,-1.794203,-0.052787,-0.214913,-0.002394,0
3,-0.821039,-0.393449,0.522915,-0.167569,-0.523426,-0.206915,2.998913,-0.100052,-0.140084,-0.385460,...,-0.121634,0.823150,-0.806277,-0.981883,-1.045483,0.090367,-0.052787,-0.214913,-0.002394,0
4,0.759467,-0.368578,-0.212203,-0.483840,-0.523426,-0.611576,-0.356094,-0.100568,-0.140171,0.417576,...,-0.121634,-1.525063,-0.806277,0.824476,1.614136,-0.333379,-0.052787,-0.214913,-0.002394,0


In [3]:
# Split in training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [4]:
# Scale
from sklearn.preprocessing import StandardScaler
X_sca = StandardScaler()
X_train = X_sca.fit_transform(X_train)
X_test = X_sca.transform(X_test)

In [5]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=7, metric='minkowski', p=1) # p = 1 is manhattan distance, p = 2 is Euclidean
clf.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=7, p=1)

In [6]:
y_pred = clf.predict(X_test)

In [7]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[66245, 11153],
       [14619, 38888]], dtype=int64)

In [8]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.86      0.84     77398
           1       0.78      0.73      0.75     53507

    accuracy                           0.80    130905
   macro avg       0.80      0.79      0.79    130905
weighted avg       0.80      0.80      0.80    130905



In [6]:
from matplotlib.colors import ListedColormap
X_set, y_set = X_train, y_train
# generates every pixel in the table. MeshGrid creates one entry for every point from X1 to X2
X1, X2 = np.meshgrid(np.arange(start=X_set[:, 0].min() - 1, stop=X_set[:, 0].max() + 1, step=0.01),
                     np.arange(start=X_set[:, 0].min() - 1, stop=X_set[:, 0].max() + 1, step=0.01))
# classifies every pixel as 0 or 1
plt.contourf(X1, X2, clf.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha=0.75, cmap=ListedColormap(('red', 'green')))
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c=ListedColormap(('red', 'green'))(i), label=j)
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
plt.title('KNN Classifier')
plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.show()

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 2 while Y.shape[1] == 49